In [1]:
import pandas as pd
import seaborn as sns

from pyDatalog import pyDatalog
from pyDatalog.pyDatalog import assert_fact, load, ask

In [2]:
path = 'AMIERules/'

In [3]:
result = pd.read_csv(path + 'FrenchRoyalty_AMIE_Rules.csv') #, delimiter='\t'
result.Body = result.Body.str.rstrip()
result.replace('>', '', regex=True, inplace=True)
result.replace('<', '', regex=True, inplace=True)
display(result.head(), result.shape)

,Body,Head,Head Coverage,Std Confidence,PCA Confidence,Positive Examples,Body size,PCA Body size,Functional variable
0,?b predecessor ?a,?a successor ?b,1.000000,1.0,1.0,471,471,471,?a
1,?b successor ?a,?a predecessor ?b,1.000000,1.0,1.0,471,471,471,?b
2,?a hasSpouse Yes,?a type Person,0.447345,1.0,1.0,994,994,994,?a
3,"?a gender ""male""@en",?a type Person,0.260126,1.0,1.0,578,578,578,?a
4,"?a gender ""female""@en",?a type Person,0.211971,1.0,1.0,471,471,471,?a


(794, 9)

## Intensional Data Base

In [4]:
amie_rule = result[['Body', 'Head']]
terms = set()
rule_list = """"""
head_dict = dict()
for d in amie_rule.values:    
#     === Creating Head of the Rule ===
    head = d[1].split('  ')
    sub, pred, obj = head[0], head[1], head[2]
    terms.add(pred)
    sub = sub.replace('?', '')
    sub = sub.capitalize()
    terms.update(sub)
    head_literal = pred+'('+sub+', '

    if obj.startswith('?'):
        obj = obj.replace('?', '')
        obj = obj.capitalize()
        terms.update(obj)
        head_literal+=obj+')'
    else:
        head_literal+="""'"""+obj+"""')"""
    rule=head_literal+""" <= """
    head_dict[head_literal]=[pred, """'"""+obj+"""'"""]
    
    
#     === Creating Body of Rule ===
    body = d[0].split('  ')
    for i in range(2, len(body), 3):
        sub, pred, obj = body[i-2], body[i-1], body[i]
        terms.add(pred)
        if sub.startswith('?'):
            sub = sub.replace('?', '')
            sub = sub.capitalize()
            terms.update(sub)
            rule+=pred+'('+sub+', '
        else:
            rule+=pred+"""('"""+sub+"""', """
        if obj.startswith('?'):
            obj = obj.replace('?', '')
            obj = obj.capitalize()
            terms.update(obj)
            rule+=obj+') & '
        else:
            rule+="""'"""+obj+"""') & """
    rule = rule[:-3]
    rule_list+='\n'+rule
# print(head_dict)

## Extensional Data base

In [5]:
path = '../KG/FrenchRoyalty/'

In [6]:
data = pd.read_csv(path + 'french_training.tsv', delimiter='\t', header=None) #
data.columns = ['s', 'p', 'o']
data.replace('>', '', regex=True, inplace=True)
data.replace('<', '', regex=True, inplace=True)
data.replace(' .', '', regex=True, inplace=True)
display(data.shape, data.head())

(10045, 3)

,s,p,o
0,1383_1385_Portuguese_interregnum,hasSpouse,No
1,1383_1385_Portuguese_interregnum,successor,John_I_of_Portugal
2,1383_1385_Portuguese_interregnum,type,Person
3,A_nor_de_Ch_tellerault,father,Aimery_I_Viscount_of_Ch_tellerault
4,A_nor_de_Ch_tellerault,gender,"""female""@en"


#### List of Terms considered in Datalog program

In [7]:
terms.update(set(data.p.unique()))
terms = ','.join(terms)
terms

'parent,F,predecessor,type,B,father,D,marriedTo,spouse,child,E,hasSpouse,A,C,successor,gender,mother,name'

In [8]:
pyDatalog.create_terms(terms)

def build_datalog_model(data):
    pyDatalog.clear()
    for d in data.values:
        # Extensional Database
        assert_fact(d[1], d[0], d[2])
    
    load("""spouse(A, B) <= parent(E, A) & predecessor(E, B)
hasSpouse(A, 'No') <= gender(A, '"male"@en')""") # rule_list
        
#         === TO DO.... ===
#     + gender(A, '<female>') <= parent(A, '""@jfg.') & successor(D, A)

In [9]:
print(rule_list)


successor(A, B) <= predecessor(B, A)
predecessor(A, B) <= successor(B, A)
type(A, 'Person') <= hasSpouse(A, 'Yes')
type(A, 'Person') <= gender(A, '"male"@en')
type(A, 'Person') <= gender(A, '"female"@en')
type(A, 'Person') <= hasSpouse(A, 'No')
parent(A, B) <= father(A, B)
parent(A, B) <= mother(A, B)
gender(A, '"male"@en') <= father(D, A) & successor(A, D)
gender(A, '"male"@en') <= child(A, D) & father(D, A)
gender(A, '"male"@en') <= father(C, A) & parent(C, A)
gender(A, '"male"@en') <= father(C, A) & predecessor(C, A)
gender(A, '"male"@en') <= father(C, A) & gender(C, '"female"@en')
gender(A, '"male"@en') <= father(C, A) & type(C, 'Person')
gender(A, '"male"@en') <= father(C, A) & hasSpouse(C, 'No')
gender(A, '"male"@en') <= father(C, A) & hasSpouse(C, 'Yes')
gender(A, '"female"@en') <= mother(D, A) & successor(A, D)
gender(A, '"female"@en') <= child(A, D) & mother(D, A)
gender(A, '"female"@en') <= mother(C, A) & predecessor(C, A)
gender(A, '"female"@en') <= mother(C, A) & type(C, '

In [10]:
def reasoning_datalog(data, head_dict):
    build_datalog_model(data)
    list_deduced_link = pd.DataFrame(columns=['s', 'p', 'o'])
#     === Query Datalog model ===
    for rule_h, val in head_dict.items():
        deduced_link = pyDatalog.ask(rule_h).answers
#         === Creating DataFrame with deduced links ===
        for i in range(len(deduced_link)):
            if len(deduced_link[i])==2:
                x = {'s': [deduced_link[i][0]], 'p': val[0], 'o': deduced_link[i][1]}
            else:
                x = {'s': [deduced_link[i][0]], 'p': val[0], 'o': val[1]}
            list_deduced_link = pd.concat([list_deduced_link, pd.DataFrame(data=x)])
#     === enriching original graph with the new links deduced ===
    graph_deduced = pd.concat([data, list_deduced_link])
    graph_deduced.drop_duplicates(keep='first', inplace=True)
    return graph_deduced, list_deduced_link

In [11]:
graph_deduced, list_deduced_link = reasoning_datalog(data, head_dict)
display(graph_deduced.shape, graph_deduced.head())

(15551, 3)

,s,p,o
0,1383_1385_Portuguese_interregnum,hasSpouse,No
1,1383_1385_Portuguese_interregnum,successor,John_I_of_Portugal
2,1383_1385_Portuguese_interregnum,type,Person
3,A_nor_de_Ch_tellerault,father,Aimery_I_Viscount_of_Ch_tellerault
4,A_nor_de_Ch_tellerault,gender,"""female""@en"


In [12]:
print(successor(A, B))

A                                                  | B                                                  
---------------------------------------------------|----------------------------------------------------
Odo_IV_Duke_of_Burgundy                            | Philip_I_Duke_of_Burgundy                          
Charles_Count_of_Valois                            | Philip_VI_of_France                                
Amadeus_VIII_Duke_of_Savoy                         | Louis_Duke_of_Savoy                                
Henry_IV_of_Castile                                | Isabella_I_of_Castile                              
Robert_II_Count_of_Artois                          | Mahaut_Countess_of_Artois                          
Leopold_III_Duke_of_Austria                        | William_Duke_of_Austria                            
Ramon_Berenguer_IV_Count_of_Provence               | Beatrice_of_Provence                               
Charles_IV_of_France                               | Ph

In [13]:
print(pyDatalog.ask("successor(A, B)"))

{('Maria_Theresa', 'Joseph_II_Holy_Roman_Emperor'), ('Francis_II_of_France', 'Charles_IX_of_France'), ('Philip_IV_of_Spain', 'John_IV_of_Portugal'), ('Louis_II_of_Anjou', 'Louis_III_of_Anjou'), ('John_Frederick_Duke_of_Brunswic_Calenberg', 'Ernest_Augustus_Elector_of_Hanover'), ('Afonso_IV_of_Portugal', 'Peter_I_of_Portugal'), ('Alfonso_V_of_Aragon', 'John_II_of_Aragon'), ('Ladislaus_the_Posthumous', 'Frederick_III_Holy_Roman_Emperor'), ('Albert_I_of_Germany', 'Henry_VII_Holy_Roman_Emperor'), ('Henry_the_Lion', 'Otto_I_Duke_of_Bavaria'), ('Victor_Amadeus_I_Duke_of_Savoy', 'Francis_Hyacinth_Duke_of_Savoy'), ('Petronilla_of_Aragon', 'Alfonso_II_of_Aragon'), ('Louis_IV_Holy_Roman_Emperor', 'Charles_IV_Holy_Roman_Emperor'), ('Sancho_VII_of_Navarre', 'Theobald_I_of_Navarre'), ('Archduchess_Anna_of_Austria', 'Renata_of_Lorraine'), ('Piero_the_Unfortunate', 'Girolamo_Savonarola'), ('Berengar_II_of_Italy', 'Otto_I_Holy_Roman_Emperor'), ('Louis_the_Pious', 'Pepin_I_of_Aquitaine'), ('Frederick_I

In [14]:
len(head_dict.keys())

15